In [2]:
import wn

# wn.download('omw:1.4')
# wn.download('odenet:1.4')
# wn.download('omw-ja:1.4')

In [179]:
wordnet = wn.Wordnet('oewn:2024')

In [180]:
from tqdm import tqdm
from collections import Counter
from utils import load_jsonl_iteratively

wordnet_root = "/data/xzhao/dataset/roman-pretrain/datasets/medical/en_jstage/tokenization"

pos2char = {'NOUN': 'n', 'VERB': 'v', 'ADJ': 'a', 'ADV': 'r'}
token_set = Counter()
for item in tqdm(load_jsonl_iteratively(f"{wordnet_root}/tokenization-0-50000.jsonl")):
    for token in item['token']['abstract']['tokens']:
        if token['pos'] in ['NOUN', 'VERB', 'ADJ', 'ADV']:
            token_set[(token['lemma'], pos2char[token['pos']])] += 1

6528it [00:06, 1026.56it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f9553ca6a40>>
Traceback (most recent call last):
  File "/home/xzhao/.pyenv/versions/adaxeval/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
26922it [00:27, 986.63it/s] 


In [183]:
lang2wn = {
    'ja': 'omw-ja:1.4',
    'zh': 'omw-cmn:1.4', 
}

def _get_word_from_synset(sense, pos, lang):
    synonyms = set()
    for s in sense.translate(lexicon=lang2wn[lang]):
        synonym = s.word().lemma()
        synonym = synonym.replace('+', '') 
        synonyms.add(synonym)
    return synonyms

def get_all_synonyms(token, synsets, pos, lang, only_first=False, more_than=None):
    syns = set()
    for i in range(len(synsets)):
        # if synsets[i].min_depth() >= 10:
        #     continue        
        for sense in synsets[i].senses():
            if lang != 'en':
                syns.update(_get_word_from_synset(sense, pos, lang))
            else:
                synonym = sense.word().lemma().replace('+', '')
                if synonym != token:
                    syns.add(synonym)

        if only_first and len(syns) > 0:
            break

        if more_than is not None and len(syns) > more_than:
            break
    return syns

lang = 'zh'
recorded = set()
for (token, pos), cnt in token_set.most_common(100):
    if token in recorded:
        continue
    recorded.add(token)
    synsets = wordnet.synsets(token, pos=pos)
    if len(synsets) == 0:
        continue
    clsym = get_all_synonyms(token, synsets, pos, lang, only_first=False, more_than=3)
    print(token, pos, clsym)    
    # break

patient n {'病人', '受事', '患者', '被动角色', '病患'}
case n {'情况', '实例', '例子', '事例', '场合'}
use v {'用', '应用', '运用', '利用', '适用', '使用'}
group n {'化学族'}
show v {'示范', '展示', '展览', '展出', '演示'}
result n {'后果', '影响', '结果', '效果'}
high a {'高的'}
study n {'书房', '报告', '主题', '领域', '题目', '学科'}
cell n {'移动电话', '手机', '电池', '蜂窝式移动电话', '大哥大', '细胞', '手提电话'}
method n {'做法', '法', '方法', '办法'}
treatment n {'治疗', '商议', '疗法', '处理'}
tumor n {'肿疡', '肿瘤', '瘤子', '瘤'}
year n {'年', '岁', '一年', '十二个月', '年度'}
day n {'太阳日', '24小时', '平均太阳日', '一昼夜', '天', '日历日', '民用日', '日', '恒星日'}
effect n {'后果', '影响', '结果', '效果'}
rate n {'费率', '率', '速率', '速度'}
low a {'低矮的', '下面的', '低垂的', '矮的', '低的'}
perform v {'做', '演出', '执行', '表演'}
level n {'层次', '程度', '水平', '度数', '度', '等级'}
also r set()
increase v {'增加', '增大', '加大', '增长'}
observe v {'察觉', '觉察', '找到', '探测', '发现', '发觉'}
cancer n {'黄道蟹属', '恶性肿瘤', '癌症'}
find v {'察觉', '觉察', '发现', '巧遇', '找到', '探测', '偶然发现', '碰巧', '发觉'}
type n set()
other a {'另外的', '其它', '其他的'}
time n {'时刻', '时钟时间', '适当时机', '时候', '时间'}
ho

In [ ]:
import wn

dog = wordnet.synsets('dog', pos='n')[0]   # "domestic dog"

print("Synset:", dog, dog.definition())

# Get its immediate hypernyms (direct superordinates)
for sup in dog.hypernyms():
    print("Hypernym:", sup, sup.definition())


Synset: Synset('omw-en-02084071-n') a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds
  Hypernym: Synset('omw-en-02083346-n') any of various fissiped mammals with nonretractile claws and typically long muzzles
  Hypernym: Synset('omw-en-01317541-n') any of various animals that have been tamed and made fit for a human environment


In [ ]:
import wn

wn.download('oewn:2021')
eng = wn.Wordnet('oewn:2021')

dog_synsets = eng.synsets('dog', pos='n')

for ss in dog_synsets:
    print(ss, ss.lexfile())
    print(ss.id, ss.definition())
    # for sense in ss.senses():
    #     print(sense._ENTITY_TYPE)


Cached file found: /home/xzhao/.wn_data/downloads/68a022474974f12bc8bdc039a26ce9a47e7286bc


Synset('oewn-02086723-n') <bound method Synset.lexfile of Synset('oewn-02086723-n')>
oewn-02086723-n a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds
Synset('oewn-10133978-n') <bound method Synset.lexfile of Synset('oewn-10133978-n')>
oewn-10133978-n a dull unattractive unpleasant girl or woman
Synset('oewn-10042764-n') <bound method Synset.lexfile of Synset('oewn-10042764-n')>
oewn-10042764-n informal term for a man
Synset('oewn-09905672-n') <bound method Synset.lexfile of Synset('oewn-09905672-n')>
oewn-09905672-n someone who is morally reprehensible
Synset('oewn-07692347-n') <bound method Synset.lexfile of Synset('oewn-07692347-n')>
oewn-07692347-n a smooth-textured sausage of minced beef or pork usually smoked; often served on a bread roll
Synset('oewn-03907626-n') <bound method Synset.lexfile of Synset('oewn-03907626-n')>
oewn-03907626-n a hinged catch that fits into a notch of a 

Skipping oewn:2021 (Open English WordNet); already added



In [120]:
ss.min_depth()

8